In [1]:
import requests
import json
import pandas as pd
import json
import requests
import scipy as sp
from datetime import datetime
from IPython.display import JSON

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# extracting the ct byks boston data frame

In [2]:
ct_bks_df = pd.read_csv(f'/Users/kamininayyar/Github_old_pc/GitHub_frm_old_pc/BostonDF.csv')

In [3]:
# checking the lat and long

lati = ct_bks_df['latitude']
longi = ct_bks_df['longitude']
print(f'{lati[10]},{longi[10]}')

42.341332,-71.076847


In [4]:
## in the below out put we see that foursquare generated entertainment locations for the above mentioned lati and longi

In [5]:
fields = 'fsq_id,name,geocodes,categories,distance,rating,hours,popularity,features'

headers = {"Accept": "application/json",  "Authorization": 'fsq3n080DfbZyBZQmDa7lBlZ1Z5ycpNJrKKFnh4ATEPok8o='}
# below query is to request the data into json
url1 = (f'https://api.foursquare.com/v3/places/search?latitude={lati}&longitude={longi}&fields={fields}&radius=1500&limit=50')
response = requests.request("GET", url1 , headers=headers).json()
resp = JSON(response)
resp

<IPython.core.display.JSON object>

In [6]:
entertainment_boston = []
for enter in response['results']:
    categories = enter.get('categories', None)
    cat_id = categories[0]['id'] if len(categories) > 0 else None
    category_name = categories[0]['name'] if len(categories) > 0 else None

    poi_boston_enter = {'lat-long': f'{lati},{longi}', 'fsq_id': enter['fsq_id'], 'cat_id': cat_id
          , 'category_name': category_name, 'categories': categories, 'name': enter.get('name', None)
           , 'rating': enter.get('rating', None), 'distance': enter.get('distance', None)
          , 'latitude': enter.get('geocodes', {}).get('main', {}).get('latitude', None)
          , 'longitude': enter.get('geocodes', {}).get('main', {}).get('longitude', None)
          , 'address': enter.get('location', {}).get('formatted_address', None)
          , 'popularity': enter.get('popularity', None), 'open_now': enter.get('hours', {}).get('open_now', None)
          
      }
      
    entertainment_boston.append(poi_boston_enter)



In [7]:
boston_entrtanmnt = pd.DataFrame(entertainment_boston)
boston_entrtanmnt.head(5)

,lat-long,fsq_id,cat_id,category_name,categories,name,rating,distance,latitude,longitude,address,popularity,open_now
0,0 42.386781\n1 42.368844\n2 42....,4e49413e81dc766f3e3d6312,16032,Park,"[{'id': 16032, 'name': 'Park', 'icon': {'prefi...",Harbour Square Park,9.3,983,43.639288,-79.378261,None,0.995420,False
1,0 42.386781\n1 42.368844\n2 42....,4ad4c062f964a520bff720e3,18025,Dance Studio,"[{'id': 18025, 'name': 'Dance Studio', 'icon':...",Fleck Dance Theater,8.3,1007,43.638298,-79.380300,None,0.934638,False
2,0 42.386781\n1 42.368844\n2 42....,4ae33054f964a520759121e3,13049,Diner,"[{'id': 13049, 'name': 'Diner', 'icon': {'pref...",Pearl Harbourfront Restaurant,8.2,1009,43.639446,-79.380716,None,0.974126,True
3,0 42.386781\n1 42.368844\n2 42....,55a19437498eeea53fa58b54,13040,Dessert Shop,"[{'id': 13040, 'name': 'Dessert Shop', 'icon':...",BeaverTails,8.5,1105,43.639722,-79.380012,None,0.990077,False
4,0 42.386781\n1 42.368844\n2 42....,597b72811b0ea5223f9f77f7,13046,Ice Cream Parlor,"[{'id': 13046, 'name': 'Ice Cream Parlor', 'ic...",Joe Bird,7.6,993,43.638513,-79.380428,None,0.963084,True


In [8]:
boston_entrtanmnt = boston_entrtanmnt.drop(['fsq_id','categories'], axis=1)


In [9]:
# save the data for future use
boston_entrtanmnt.to_csv(f'/Users/kamininayyar/Github_old_pc/GitHub_frm_old_pc/boston_entrtanmnt_fsq.csv', index = False)

# yelp api

In [10]:
from IPython.display import JSON

In [11]:
api_key='Ih7ZOj3LbZz6kpbzENfPvbjh0l4oVSDMUf4SbLPJSCIUueExyhoiwk9Fm6dRmXCL59D3Bvi-Ok57nILnlXGPIvW-V-19R9nbqjV7fL4F2N-28gQrWlNy2FGAHB-NY3Yx'
headers = {'Authorization': 'Bearer %s' % api_key}

In [12]:
url='https://api.yelp.com/v3/businesses/search?latitude=42.341332&longitude=-71.076847&radius=1000&limit=50'
# I had to use specific lat and long because seems like ylp URL can't take the dictionary of lat and long as four square did

response1 =requests.get(url, headers=headers)

In [13]:
json.loads(response1.text)
data = json.loads(response1.text)

In [14]:
df = pd.json_normalize(data)
df

,businesses,total,region.center.longitude,region.center.latitude
0,"[{'id': 'NYa-JphaaB41ElGsb3iawA', 'alias': 'to...",431,-71.076847,42.341332


In [15]:
seperating_sites = []
for sites in data['businesses']:
    all_sites = {
          'lat-long': f'{lati},{longi}', 'category_name': sites['categories'][0]['alias'], 'name': sites['name']
          , 'distance': sites['distance'], 'latitude': sites['coordinates']['latitude']
          , 'longitude': sites['coordinates']['longitude'], 'address': sites['location']['display_address']
          , 'is_closed': sites.get('is_closed'), 'rating': sites.get('rating')
          , 'rating_count': sites.get('review_count')
         
      }
      
    seperating_sites.append(all_sites)
    

In [16]:
boston_ylp_df = pd.DataFrame(seperating_sites)
boston_ylp_df.head()

,lat-long,category_name,name,distance,latitude,longitude,address,is_closed,rating,rating_count
0,0 42.386781\n1 42.368844\n2 42....,tapas,Toro,488.960833,42.336988,-71.075924,"[1704 Washington St, Boston, MA 02118]",False,4.0,2425
1,0 42.386781\n1 42.368844\n2 42....,bars,The Friendly Toast,833.597293,42.348348,-71.073273,"[35 Stanhope St, Back Bay, MA 02116]",False,4.5,2533
2,0 42.386781\n1 42.368844\n2 42....,seafood,Atlantic Fish,945.200651,42.349222,-71.081127,"[761 Boylston St, Boston, MA 02116]",False,4.0,2967
3,0 42.386781\n1 42.368844\n2 42....,newamerican,The Salty Pig,622.001099,42.346900,-71.076121,"[130 Dartmouth St, Boston, MA 02116]",False,4.0,1813
4,0 42.386781\n1 42.368844\n2 42....,bakeries,Flour Bakery + Cafe,369.029836,42.338330,-71.074880,"[1595 Washington St, Boston, MA 02118]",False,4.0,920


In [17]:
boston_ylp_df.to_csv(f'/Users/kamininayyar/Github_old_pc/GitHub_frm_old_pc/boston_ylp_df.csv', index = False)

What is the different observation between two api's?

# Major difference between four square gave entertainment and restraunts where as yelp only gave restraunts

Top 10 venues from both api's

In [18]:
print(boston_ylp_df.iloc[:, [2,8]].sort_values('rating',ascending=False).head(10))

                   name  rating
49  Mana Escondido Café     4.5
10    Lobstah On A Roll     4.5
26     KAVA neo-taverna     4.5
1    The Friendly Toast     4.5
31          anoush'ella     4.5
22   The Capital Grille     4.5
19               Blunch     4.5
13                 FoMu     4.5
40                GRECO     4.5
11           SRV Boston     4.5


In [19]:
print(boston_entrtanmnt.iloc[:, [3,4]].sort_values('rating',ascending=False).head(10))

                         name  rating
0         Harbour Square Park     9.3
9             Toronto Islands     9.3
31            Roundhouse Park     9.3
6         Harbourfront Centre     9.1
5                        Miku     9.0
26           Scotiabank Arena     8.7
29          Maple Leaf Square     8.6
23  Air Canada Signature Club     8.6
25        Real Sports Apparel     8.6
3                 BeaverTails     8.5
